In [1]:
import os
import cv2 as cv
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.utils import to_categorical
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [3]:
path = "/kaggle/input/covid19-radiography-dataset/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset"
images = []
labels = []
i = 0
for files in os.listdir(path):
    counter = 0
    for imagesPath in os.listdir(path+"/"+files):
        if counter==1330:
            break
        img = cv.imread(path+"/"+files+"/"+imagesPath)
        img = cv.resize(img,(100,100))
        img = img/255
        images.append(img)
        labels.append(i)
    i += 1

In [4]:
len(labels)

21165

In [5]:
# labels = to_categorical(labels)

In [6]:
x_train,x_test,y_train,y_test = train_test_split(images,labels,test_size=0.2,random_state=42,shuffle=True)

In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [8]:
import numpy as np
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the custom SkipConnection layer
class SkipConnection(layers.Layer):
    def __init__(self, num, kernel, **kwargs):
        super(SkipConnection, self).__init__(**kwargs)
        self.num = num
        self.kernel = kernel
        # Define the Conv2D layers
        self.conv1 = layers.Conv2D(num, (kernel, kernel), padding="same")
        self.batchnorm1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(num, (kernel, kernel), padding="same")
        self.batchnorm2 = layers.BatchNormalization()
        # self.conv3 = layers.Conv2D(num, (kernel, kernel), activation="relu", padding="same")
        self.drop = layers.Dropout(0.2)
        # Define a Conv2D layer for the skip path to match dimensions
        self.match_channels = layers.Conv2D(num, (1, 1), activation=None, padding="same")
        self.add = layers.Add()

    def call(self, inputs):
        # Save the input for skip connection
        input_skip = self.match_channels(inputs)
        # Apply convolutional layers
        x = self.conv1(inputs)
        x = self.batchnorm1(x)
        x = layers.ReLU()(x)
        x = self.drop(x)
        x = self.conv2(x)
        x = self.batchnorm2(x)

        x = self.add([x, input_skip])
        x = layers.ReLU()(x)
        return x

In [10]:
tf.keras.backend.clear_session()

In [11]:
# physical_devices = tf.config.list_physical_devices('GPU')
# for device in physical_devices:
#     try:
#         tf.config.experimental.set_memory_growth(device, True)
#     except RuntimeError as e:
#         print(e)  # Handle any errors (e.g., if memory growth was already set)

In [12]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model = tf.keras.Sequential()
model.add(layers.Input(shape=(100,100,3)))  # Example input shape

listSteps = [32,64]
for step in listSteps:
    model.add(SkipConnection(num=step, kernel=5))  # Use the correct parameter name

model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.2))
# model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))  # Example output layer for 10 classes

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])
History = model.fit(x_train,y_train, epochs=100, batch_size=64, validation_data=(x_test,y_test),callbacks=[early_stopping])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ skip_connection                 │ (None, 100, 100, 32)   │        28,448 │
│ (SkipConnection)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ skip_connection_1               │ (None, 100, 100, 64)   │       156,352 │
│ (SkipConnection)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 640000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │     2,560,004 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,744,804 (10.47 MB)

 Trainable params: 2,744,420 (10.47 MB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/100


I0000 00:00:1734556265.371039      65 service.cc:145] XLA service 0x78c0e4006a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734556265.371145      65 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-12-18 21:11:15.803005: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,32,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,32,100,100]{3,2,1,0}, f32[64,64,100,100]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-12-18 21:11:16.060817: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.257933176s
Trying algorithm eng0{} for conv

  2/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 0.3242 - loss: 14.6268  

I0000 00:00:1734556280.973406      65 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


265/265 ━━━━━━━━━━━━━━━━━━━━ 59s 151ms/step - accuracy: 0.5687 - loss: 20.0665 - val_accuracy: 0.5575 - val_loss: 1.1020
Epoch 2/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.7582 - loss: 0.6301 - val_accuracy: 0.7245 - val_loss: 0.6836
Epoch 3/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.8291 - loss: 0.4549 - val_accuracy: 0.7489 - val_loss: 0.6963
Epoch 4/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.8566 - loss: 0.3777 - val_accuracy: 0.7682 - val_loss: 0.7433
Epoch 5/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.8837 - loss: 0.3111 - val_accuracy: 0.8037 - val_loss: 0.5366
Epoch 6/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.9031 - loss: 0.2557 - val_accuracy: 0.7931 - val_loss: 0.6013
Epoch 7/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.9274 - loss: 0.1948 - val_accuracy: 0.6811 - val_loss: 1.2730
Epoch 8/100
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.9497 - loss: 0.1448 - va

In [13]:
best_epoch = len(History.history['val_loss']) - early_stopping.patience
best_val_accuracy = max(History.history['val_accuracy'])

print(f"Best Epoch: {best_epoch}")
print(f"Best Validation Accuracy: {best_val_accuracy}")

Best Epoch: 5
Best Validation Accuracy: 0.8036853075027466


In [14]:

# strategy = tf.distribute.MirroredStrategy()
# print("Number of devices: ", strategy.num_replicas_in_sync)
# with strategy.scope():
#     model = tf.keras.Sequential()
#     model.add(layers.Input(shape=(200, 200, 3)))  # Example input shape
    
#     listSteps = [128,256]
#     for step in listSteps:
#         model.add(SkipConnection(num=step, kernel=3))  # Use the correct parameter name
    
#     model.add(layers.Flatten())
#     # model.add(layers.Dense(128, activation='relu'))
#     # model.add(layers.Dropout(0.05))
#     # model.add(layers.Dense(64, activation='relu'))
#     model.add(layers.Dense(4, activation='softmax'))  # Example output layer for 10 classes
    
#     model.summary()
    
#     model.compile(loss='binary_crossentropy',
#                   optimizer=Adam(),
#                   metrics=['accuracy'])
# model.fit(x_train,y_train, epochs=50, batch_size=92, validation_data=(x_test,y_test))

In [15]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(16932, 100, 100, 3)
(4233, 100, 100, 3)
(16932, 4)
(4233, 4)


In [16]:
# History = model.fit(x_train,y_train, epochs=50, batch_size=46, validation_data=(x_test,y_test))